In [2]:
import os
from PIL import Image
import numpy as np

# --- Configuration ---
input_folder = 'images'       # folder containing your grayscale or color images
output_folder = 'binary_masks'      # folder where binary images will be saved
threshold = 80                     # threshold value (0–255)

# --- Ensure output folder exists ---
os.makedirs(output_folder, exist_ok=True)

# --- Loop through all image files ---
for filename in os.listdir(input_folder):
    # Process only common image types
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        input_path = os.path.join(input_folder, filename)

        # Split filename into name and extension, insert '_m2', and rejoin
        name, ext = os.path.splitext(filename)
        output_filename = f"{name}.png" # Added .png extension
        output_path = os.path.join(output_folder, output_filename)

        # Load image and convert to grayscale
        gray = Image.open(input_path).convert('L')
        gray_array = np.array(gray)

        # Apply threshold
        binary_array = (gray_array > threshold) * 255

        # Convert to PIL image
        binary_image = Image.fromarray(np.uint8(binary_array))

        # Save result
        binary_image.save(output_path)

        print(f"Saved binary mask: {output_path}")

print("✅ All images processed and saved to:", output_folder)

Saved binary mask: binary_masks/sem0002.png
Saved binary mask: binary_masks/sem0001.png
✅ All images processed and saved to: binary_masks


In [3]:
import os
os.makedirs("svg_from_binary_masks", exist_ok=True)
print("Directory 'svg_from_binary_masks' ensured to exist.")

Directory 'svg_from_binary_masks' ensured to exist.


In [5]:
import sys
if 'svgwrite' not in sys.modules:
  !pip install svgwrite

import os
from PIL import Image
import numpy as np
from skimage import measure
import svgwrite

# Re-define mask_to_svg function (from cell hV9SQFhrzA7h and 56bbef00)
def mask_to_svg(mask, output_svg_path):
    contours = measure.find_contours(mask, 0.5)
    dwg = svgwrite.Drawing(output_svg_path, profile='tiny')
    for contour in contours:
        # `find_contours` returns (row, col) coordinates, which correspond to (y, x)
        # SVG paths expect (x, y)
        points = [(float(x), float(y)) for y, x in contour]
        if points:
            dwg.add(dwg.polygon(points, fill='none', stroke='red', stroke_width=1))
    dwg.save()

# 1. Define input and output directories
input_binary_dir = 'binary_masks'
output_svg_dir = 'svg_from_binary_masks'

# 2. Get a list of all .png files within the binary_masks directory
binary_mask_files = [f for f in os.listdir(input_binary_dir) if f.lower().endswith('.png')]

print(f"Found {len(binary_mask_files)} binary mask files in '{input_binary_dir}'.")

# 3. Loop through each .png file
for filename in binary_mask_files:
    # a. Construct the full path to the input binary mask image
    input_path = os.path.join(input_binary_dir, filename)

    # b. Create an appropriate output SVG filename
    name, ext = os.path.splitext(filename)
    output_svg_filename = f"{name}.svg"

    # c. Construct the full path for the output SVG file
    output_svg_path = os.path.join(output_svg_dir, output_svg_filename)

    # d. Load the binary mask image and convert to NumPy array
    # Ensure the image is in grayscale ('L') mode as mask_to_svg expects a 2D array
    binary_image_pil = Image.open(input_path).convert('L')
    binary_mask_np = np.array(binary_image_pil)

    # e. Call the mask_to_svg function
    mask_to_svg(binary_mask_np, output_svg_path)

    # f. Print a confirmation message
    print(f"Generated SVG for {filename}: {output_svg_path}")

print("Finished processing all binary masks and generating SVG files.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 2.4 MB/s eta 0:00:00
Found 2 binary mask files in 'binary_masks'.
Generated SVG for sem0002.png: svg_from_binary_masks/sem0002.svg
Generated SVG for sem0001.png: svg_from_binary_masks/sem0001.svg
Finished processing all binary masks and generating SVG files.
